<a href="https://colab.research.google.com/github/ravi07bec/models/blob/master/Reccos_FG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Getting the Data
 !curl -L -o myfile.csv "https://drive.google.com/file/d/14ONwZLp3xsfktetefOENAQDbk-5ATGjb/view"
 #!wget --no-check-certificate https://drive.google.com/a/iima.ac.in/uc?id=14ONwZLp3xsfktetefOENAQDbk-5ATGjb&export=download' -O FILENAME.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  133k    0  133k    0     0   364k      0 --:--:-- --:--:-- --:--:--  363k


In [0]:
import pandas as pd
import numpy as np
import time
#import turicreate as tc
from sklearn.model_selection import train_test_split
from datetime import datetime


In [0]:
import os
from google.colab import drive
#drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Notebooks_Medium/Music/')
!ls "/content/gdrive/My Drive/Notebooks_Medium/Music/"



 BabyElephantWalk60.wav
 Datathon_sample_data.csv
'knock (1).wav'
'Mahabharat flute theme - simple yet so beautiful.wav'
 Music.ipynb
'One breath challenge - can you play this.mp3'
'One breath challenge - can you play this.wav'
 Reccos_FG.ipynb


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Notebooks_Medium/Music/Datathon_sample_data.csv')

In [0]:
import gzip
import gensim 
import logging
df=df.sample(frac=1)

In [0]:
sample=pd.DataFrame(df['customer_id'].unique()[1:5000])
sample.columns=['customer_id']
df2=pd.merge(sample,df)
print(df2.head(),df2.shape)


   customer_id  date_key  store_key  till_no  product_key  sale_trans_no  \
0    175003652  20180902       6289        2      1307211         271323   
1    175003652  20180728       6289        1      1850694          50896   
2    175003652  20180608       6289        1       496668          46013   
3    175003652  20180815       6289        4     28509314          12785   
4    175003652  20181012       8645        2     25383859          84406   

   sale_tot_qty  
0           2.0  
1           0.5  
2           2.0  
3           1.0  
4           1.0   (1045260, 7)


In [0]:
# In[67]:


train_data=df2
#train_data=train_data.sample(frac=1)

train_data['date'] = pd.to_datetime(train_data['date_key'], format='%Y%m%d')
display(train_data.head())
test_data = train_data[train_data['date']>'2018-09-30']
display(test_data.head())
train_data['product_key'] = train_data['product_key'].astype(str)
train_data = train_data.groupby('customer_id')['product_key'].apply(list).reset_index()


# In[68]:

documents = []


# In[69]:

documents = list(train_data['product_key'])


# In[106]:

# build vocabulary and train model
model = gensim.models.Word2Vec(
        documents,
        size=150,
        window=10,
        min_count=1)


# In[107]:



,customer_id,date_key,store_key,till_no,product_key,sale_trans_no,sale_tot_qty,date
0,175003652,20180902,6289,2,1307211,271323,2.0,2018-09-02
1,175003652,20180728,6289,1,1850694,50896,0.5,2018-07-28
2,175003652,20180608,6289,1,496668,46013,2.0,2018-06-08
3,175003652,20180815,6289,4,28509314,12785,1.0,2018-08-15
4,175003652,20181012,8645,2,25383859,84406,1.0,2018-10-12


,customer_id,date_key,store_key,till_no,product_key,sale_trans_no,sale_tot_qty,date
4,175003652,20181012,8645,2,25383859,84406,1.0,2018-10-12
9,175003652,20181009,6289,4,519829,22859,1.0,2018-10-09
14,175003652,20181009,6289,4,939362,22859,1.0,2018-10-09
31,175003652,20181013,6289,2,496004,280403,1.0,2018-10-13
39,175003652,20181009,8645,1,26221717,55715,1.0,2018-10-09


In [0]:
model.train(documents,total_examples=len(documents),epochs=10)


# In[73]:



(10012362, 10452600)

In [0]:
model.vocabulary


# In[108]:

#print(model.wv[train_data['product_key'][1]])


# In[109]:

word_vectors = model.wv


# In[110]:

np.mean(word_vectors.wv[train_data['product_key'][1]], axis=0)


# In[111]:

customer_persona = []


# In[112]:

for i in range(len(train_data['product_key'])):
    customer_persona.append(np.mean(model.wv[train_data['product_key'][i]], axis=0))


# In[117]:

w1 = customer_persona[1]


# In[118]:

w1


# In[121]:

top10 = model.wv.similar_by_vector(w1)


# In[128]:

top10[1][0]


# In[129]:

recommendations = []


# In[130]:

for vec in customer_persona:
    top10 = model.wv.similar_by_vector(vec)
    recco = []
    for i in range(len(top10)):
        recco.append(top10[i][0])
    recommendations.append(recco)


# In[132]:

recommendations


# In[134]:

final_preds = pd.DataFrame({'recommendations': recommendations, 'customer_id': train_data['customer_id']})


# In[135]:

final_preds.head()


# In[136]:

test_data.head()


# In[137]:

# Perform a groupby and apply a string join.
test_data['product_key'] = test_data['product_key'].astype(str)


# In[138]:

test_data = test_data.groupby('customer_id')['product_key'].apply(list).reset_index()


# In[139]:

test_data.head()


# In[140]:

final = final_preds.merge(test_data)


# In[141]:

final.head()


# In[142]:

final['recommendations'][1]


# In[145]:

list(set(final['recommendations'][3]).intersection(final['product_key'][3]))


# In[146]:

def common_prod(x):    
    return list(set(x['recommendations']).intersection(x['product_key']))  


# In[147]:

final['common_overall'] = final.apply(common_prod, axis=1)

print(final.head())

# In[151]:

def recall_recco(x):
    return (len(x['common_overall'])/len(x['product_key']))


# In[156]:

def recall_recco_10(x):
    return (len(x['common_overall'])/len(x['recommendations']))


# In[165]:

def recall_overall(x):
    return (min(len(x['product_key']),10))


# In[168]:

def recall_common_overall(x):
    return (len(x['common_overall']))


# In[152]:

final['recall'] = final.apply(recall_recco, axis=1)


# In[157]:

final['recall_10'] = final.apply(recall_recco_10, axis=1)


# In[158]:

final['recall'].mean()*100


# In[159]:

final['recall_10'].mean()*100


# In[166]:

final['recall_overall'] = final.apply(recall_overall, axis=1)


# In[169]:

final['recall_common_overall'] = final.apply(recall_common_overall, axis=1)


# In[170]:
print(final.head())



# In[171]:

print(100*final['recall_common_overall'].sum() / final['recall_overall'].sum())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   customer_id                                    recommendations  \
0    116201447  [1829943, 495595, 535076, 505395, 1161910, 153...   
1    116282372  [1377640, 23583348, 1879646, 29218215, 2150513...   
2    116768700  [27307545, 1563268, 1479998, 21292799, 1861941...   
3    116850091  [513935, 2644330, 20174996, 2674021, 1822915, ...   
4    116953622  [1380821, 22520230, 22338826, 507412, 24054339...   

                                         product_key       common_overall  
0  [2691862, 21920915, 27801300, 530341, 28477625...                   []  
1  [525170, 22366435, 1214329, 496667, 1377640, 5...  [1377640, 23583348]  
2  [22328464, 26911275, 25410497, 28802170, 26911...            [1479998]  
3  [521924, 1758433, 1849704, 525420, 1848934, 21...                   []  
4  [28403330, 1419769, 521516, 1900274, 23511838,...                   []  
   customer_id                                    recommendations  \
0    116201447  [1829943, 495595, 535076, 505395, 1161910, 1

In [0]:
final

,customer_id,recommendations,product_key,common_overall,recall,recall_10,recall_overall,recall_common_overall
0,116201447,"[535076, 495595, 1842026, 1869868, 29730629, 2...","[['2691862', '21920915', '27801300', '530341',...",[],0.0,0.0,1,0
1,116282372,"[1377640, 23583348, 21505133, 28609616, 240278...","[['525170', '22366435', '1214329', '496667', '...",[],0.0,0.0,1,0
2,116432241,"[21909980, 1850064, 1850187, 21910057, 1849127...","[['1849088', '1850198', '21910126', '1850198',...",[],0.0,0.0,1,0
3,116768700,"[2703591, 21292799, 28136324, 1228769, 2729265...","[['22328464', '26911275', '25410497', '2880217...",[],0.0,0.0,1,0
4,116850091,"[513935, 507394, 1850709, 2630295, 1361348, 26...","[['521924', '1758433', '1849704', '525420', '1...",[],0.0,0.0,1,0
5,116940805,"[21641331, 1532669, 519911, 1842026, 1041635, ...","[['1389898', '1327141', '28901476', '511429', ...",[],0.0,0.0,1,0
6,116949170,"[1882382, 1882449, 1882456, 23959897, 1882383,...","[['24412871', '1877694', '1214385', '1374004',...",[],0.0,0.0,1,0
7,116953622,"[525170, 1426455, 22338826, 1793451, 22694021,...","[['28403330', '1419769', '521516', '1900274', ...",[],0.0,0.0,1,0
8,116957948,"[499909, 24645199, 24645181, 29590936, 496047,...","[['1663267', '496116', '1850709', '496004', '3...",[],0.0,0.0,1,0
9,116973681,"[1793451, 20927132, 1869868, 496451, 1850694, ...","[['1487786', '29485667', '522110', '29610027',...",[],0.0,0.0,1,0
